In [1]:
import tweepy
from openpyxl import load_workbook, Workbook

### Part 1
Collect the tweets

In [2]:
from auth import consumer_key, consumer_secret

In [3]:
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)

In [4]:
api = tweepy.API(auth)

In [5]:
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)

In [6]:
tweet_text = set()

In [7]:
for status in limit_handled(
    tweepy.Cursor(api.search, tweet_mode="extended", q='vaccine OR vaccination -filter:retweets', count=10, lang='en').items(5)):
        try:
            tweet_text.add(status.full_text)
        except AttributeError:
            tweet_text.add(status.text)
            print("TEXT ONLY")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: generator 'limit_handled' raised StopIteration
  


### Part 2
Save the tweets

In [8]:
loc = './tweets.xlsx'

In [9]:
try:
    wb = load_workbook(loc)
    ws = wb.active
except FileNotFoundError:
    wb = Workbook()
    ws = wb.active
    ws['A1'] = "Tweets"
    ws['B1'] = "Relevant"
    ws['C1'] = 'Positive'
    ws['D1'] = 'Negative'
    ws['E1'] = 'Neutral'

In [10]:
saved_text = set()

In [11]:
for row in ws.iter_rows(min_row=2, max_col=1, max_row=ws.max_row):
    for cell in row:
        saved_text.add(cell.value)

In [12]:
new_text = set.difference(tweet_text, saved_text)

In [13]:
rows = ws.max_row + 1
for item, text in enumerate(new_text):
    ws.cell(row=item+rows, column=1, value=text)

In [14]:
wb.save(loc)
wb.close()

### References
Official:   https://developer.twitter.com/en/docs/tweets/search/guides/standard-operators

Tweepy:     http://docs.tweepy.org/en/latest/api.html#help-methods

xlsxwriter: https://xlsxwriter.readthedocs.io/workbook.html

xlrd:       https://www.geeksforgeeks.org/reading-excel-file-using-python/

###### Recommended approach for writing in Excel
openpyxl:   https://openpyxl.readthedocs.io/en/stable/tutorial.html
            